In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [4]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
#full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [5]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
#choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
#frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [15]:
def choose_z_score_for_MDMpred(curr_z,lb,ub,data, collection):
    
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    eps = 0.001

    model = gp.Model('mdm_z_value')
    model.setParam('OutputFlag', 0)
    lam = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0.0, name="lam")
    x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="x")
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta")

    # Add constraints
    for j in range(m):
        model.addConstr(sum(x[i, j] for i in collection[j]) == 1, name=f"normalization_{j}")

    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                model.addConstr(x[j, i] == 0, name=f"forcing_zero_{i}_{j}")
    
    for i in range(m):
        model.addConstr(lam[i] <= 1, name=f"lam_bound_{i}")

    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"pro_{i}_aspair_{i}_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"ppro_{i}_aspair_{j}_{k}")
    ''' for i in range(n):
        for j in range(m):
            if data[i, j] !=0:
                model.addConstr(x[i,j]>= lb[i,j])
                model.addConstr(x[i,j]<= ub[i,j]) '''

    for j in range(len(collection)):
        for i in collection[j]:
            model.addConstr(x[i,j]>= lb[i,j])
            model.addConstr(x[i,j]<= ub[i,j])

    model.setObjective(0, GRB.MINIMIZE)

    # Set Gurobi parameters if needed
    model.optimize()

    if model.Status == GRB.OPTIMAL :
        print('Optimal solution found!')
        print(f'The z value for MDM prediction is {curr_z}')
        return [1, model.Runtime]
    else:
        return [-1, model.Runtime]

In [7]:
## RUM limit formulation
def find_element_permutation_indexes(n, partial_lists):
    perm_collection = []
    
    elements = list(range(n))
    perms = list(permutations(elements))
    #index_permutations = {perm: idx for idx, perm in enumerate(perms)}
    result = {}

    for partial_list in partial_lists:
        partial_result = {}
        perm_element = []
        for element in partial_list:
            element_indexes = [idx for idx, perm in enumerate(perms) if perm.index(element) == min(perm.index(e) for e in partial_list)]
            partial_result[element] = element_indexes
            perm_element.append(element_indexes)
        result[tuple(partial_list)] = partial_result
        perm_collection.append(perm_element)
    return result,perm_collection

def find_product_notin_assortment(collection,n):
    notin_collection = []
    for i in range(len(collection)):
        notin_assortment = []
        for j in range(n):
            if j not in collection[i]:
                notin_assortment.append(j)
        notin_collection.append(notin_assortment)
    return notin_collection


In [8]:
def choose_z_score_for_RUMpred(curr_z,lb,ub,data,collection,perm_collection,produts_notin_collection):
    n,m = data.shape # number of products
    fact = math.factorial(n) # number of permutations
    
    model = gp.Model('rum_z_value') #define a model
    model.setParam('OutputFlag', 0)
    
    # define nonnegative continuous variables for lambda, the probability of each permutation
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam') 
    # define nonnegative continuous variables for choice probability of each product in each assortment
    x = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0.0, name="x") 
    
    #normalization constraint of lambda
    model.addConstr(sum(lam[i] for i in range(fact))==1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        # normalization constraint for each assortment. 
        #model.addConstr(gp.quicksum(x[k, i] for k in collection[i]) == 1)
        for j in range(len(collection[i])):
            # constraints for RUM choice probabilities 
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- x[collection[i][j],i]==0)
            
            # constraints for the lower bound and upper bound values
            model.addConstr(x[collection[i][j],i]>= lb[collection[i][j],i])
            model.addConstr(x[collection[i][j],i]<= ub[collection[i][j],i])
        
        for k in produts_notin_collection[i]:
            # constraints for products that are not in the assortments
            model.addConstr(x[k,i] == 0)
    
    model.setObjective(0, sense=gp.GRB.MINIMIZE)
    #model.setObjective(0)
    model.optimize()
    status = model.status
    print("Gurobi optimization status:", status)
    # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        print('Optimal solution found!')
        print(f'The z value for RUM prediction is {curr_z}')
        x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
        x_values = np.array(x_values)
        return [1,x_values,model.Runtime]
    else:
        print('No solution found or an error occurred.')
        return [-1]
    

In [9]:
stardard_error_20 = np.zeros(frequency_collection_20.shape)
for i in range(stardard_error_20.shape[0]):
    for j in range(stardard_error_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            stardard_error_20[i][j] = np.sqrt((1-choice_collection_20[i][j])/frequency_collection_20[i][j])

In [18]:
## MDM limit formulation
def row_assortment_data(data,collection):
    pre_data = copy.deepcopy(data)
    
    for j in range(data.shape[0]):
        for i in range(len(collection)):
            if j not in collection[i]:
                pre_data[j][i] = -1
    
    return pre_data

def find_indexes_nonnegative_descending(arr):
    row_indexes_sorted_desc_nonnegative = []
    row_indexes_positive_same_value = []

    for row in arr:
        # Find indexes with non-negative values
        nonnegative_row = row[row >= 0]  # Consider only non-negative values
        unique_elements, unique_indexes = np.unique(nonnegative_row, return_index=True)
        sorted_indexes_desc = np.nonzero(row >= 0)[0][unique_indexes][np.argsort(-unique_elements)]

        row_indexes_sorted_desc_nonnegative.append(sorted_indexes_desc)

        # Keep all indexes of elements with positive and same values
        positive_values = unique_elements[unique_elements > 0]
        positive_same_value_indexes = [np.where(row == val)[0] for val in positive_values if np.count_nonzero(row == val) > 1]
        row_indexes_positive_same_value.append(positive_same_value_indexes)

    return row_indexes_sorted_desc_nonnegative, row_indexes_positive_same_value


In [46]:
def mdm_limit_l_infinity(data, collection):
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    eps = 0.01

    model = gp.Model()
    model.setParam('OutputFlag', 0)
    lam = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0.0, name="lam")
    x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="x")
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta")
    z = model.addVar(name="z")

    
    # Define auxiliary binary variables for absolute value terms
    abs_vars = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="abs")
    ''' abs_vars = {}
    for i in range(n):
        for j in range(m):
            abs_vars[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"abs_{i}_{j}") '''

    # Add constraints
    for j in range(m):
        model.addConstr(sum(x[i, j] for i in range(n)) == 1, name=f"normalization_{j}")

    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                model.addConstr(x[j, i] == 0, name=f"forcing_zero_{i}_{j}")
    
    for i in range(m):
        model.addConstr(lam[i] <= 1, name=f"lam_bound_{i}")

    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"pro_{i}_aspair_{i}_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"ppro_{i}_aspair_{j}_{k}")

    # Add linear constraints to represent absolute value terms
    ''' for i in range(n):
        for j in range(m):
            model.addConstr(x[i, j] - data[i, j] <= abs_vars[i, j], name=f"abs_const1_{i}_{j}")
            model.addConstr(-x[i, j] + data[i, j] <= abs_vars[i, j], name=f"abs_const2_{i}_{j}")
            model.addConstr(abs_vars[i, j] <=z, name="l_inf_norm_upper_bound") '''
    for i in range(n):
        for j in range(m):
            model.addConstr(x[i, j] - data[i, j] <= z, name=f"abs_const1_{i}_{j}")
            model.addConstr(-x[i, j] + data[i, j] <= z, name=f"abs_const2_{i}_{j}")
            #model.addConstr(abs_vars[i, j] <=z, name="l_inf_norm_upper_bound")
            
    # Define the modified objective function using auxiliary variables
    #obj = (sum(data[i, j] * abs_vars[i, j] for i in range(n) for j in range(m)))

    model.setObjective(z, GRB.MINIMIZE)

    # Set Gurobi parameters if needed
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        if model.objVal < math.pow(10, -6):
            return [0, model.Runtime]
        else:
            print('mdm l-infinity norm radius',z.x)
            model.write("mdmlimit_out.sol")
            model.write("mdmlimit_model.lp") 
            return [model.objVal, model.Runtime]
    else:
        print('mdm limit formulation issue')


In [47]:
mdm_limit_l_infinity(choice_collection_20,collection_20)

mdm l-infinity norm radius 0.08333333333333331


[0.08333333333333331, 0.6368470191955566]

In [51]:
def rum_limit_l_infinity(data,collection,perm_collection,produts_notin_collection):
    
    n,m = data.shape # number of products
    fact = math.factorial(n) # number of permutations
    
    model = gp.Model('rum') #define a model
    model.setParam('OutputFlag', 0)
    
    # define nonnegative continuous variables for lambda, the probability of each permutation
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam') 
    # define nonnegative continuous variables for choice probability of each product in each assortment
    x = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0, name="x") 
    # define the variables for absolute value
    y = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0, name="y")
    z = model.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="z")
    #normalization constraint of lambda
    model.addConstr(sum(lam[i] for i in range(fact))==1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        # normalization constraint for each assortment. 
        model.addConstr(gp.quicksum(x[k, i] for k in collection[i]) == 1)
        for j in range(len(collection[i])):
            # constraints for RUM choice probabilities 
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- x[collection[i][j],i]==0)
            # constraints for the absolute value
            ''' model.addConstr(x[collection[i][j],i] - data[collection[i][j]][i] - y[collection[i][j],i] <=0)
            model.addConstr(data[collection[i][j]][i] - x[collection[i][j],i] -  y[collection[i][j],i] <=0) '''
            
            model.addConstr(x[collection[i][j],i] - data[collection[i][j]][i] - z <=0)
            model.addConstr(data[collection[i][j]][i] - x[collection[i][j],i] -  z <=0)
        for k in produts_notin_collection[i]:
            # constraints for products that are not in the assortments
            model.addConstr(x[k,i] == 0)
    
    #model.setObjective(gp.quicksum(offer_times[j]*data[i][j] * y[i, j] for i in range(n) for j in range(m)/sum(offer_times)), sense=gp.GRB.MINIMIZE)
    #model.setObjective(0)
    # Define the modified objective function using auxiliary variables
    #obj = (sum(offer_times[j]*data[i, j] *  y[i, j]  for i in range(n) for j in range(m)))
    #obj = (sum(offer_times[j]*data[i][j] * abs_vars[i, j] for i in range(n) for j in range(m))/sum(offer_times))
    obj = (z)

    model.setObjective(obj, GRB.MINIMIZE)
    
    model.optimize()
   
    # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        print('Optimal solution found!')
        print(f'L-infinity Norm Loss (Objective Value) = {model.objVal}')
        x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
        x_values = np.array(x_values)
            
        return [model.objVal,model.Runtime]
    else:
        print('No solution found or an error occurred.')
       
    
    

In [52]:
element_indexes_20,perm_collection_20 = find_element_permutation_indexes(n, collection_20)
products_notin_collection_20 = find_product_notin_assortment(collection_20,n)
rum_limit_l_infinity(choice_collection_20,collection_20,perm_collection_20,products_notin_collection_20)


Optimal solution found!
L-infinity Norm Loss (Objective Value) = 0.07225609756097562


[0.07225609756097562, 3.6576449871063232]

In [60]:
confidence_level = [0.99,0.985]
alpha = np.ones(len(confidence_level)) - confidence_level
# Find z-score for the given confidence level
z_score = stats.norm.ppf(1 - alpha / 2)
print(f'the testing z values are {z_score}')

stardard_error_20 = np.zeros(frequency_collection_20.shape)
for i in range(stardard_error_20.shape[0]):
    for j in range(stardard_error_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            stardard_error_20[i][j] = np.sqrt((1-choice_collection_20[i][j])/frequency_collection_20[i][j])

the testing z values are [2.5758293  2.43237906]


In [61]:
# test for offertimes=20
target_z_rum = 0
for curr_z in z_score:
    lb_20 = np.zeros(frequency_collection_20.shape)
    ub_20 = np.zeros(frequency_collection_20.shape)
    for i in range(lb_20.shape[0]):
        for j in range(lb_20.shape[1]):
            if frequency_collection_20[i][j]!=0:
                lb_20[i][j] = choice_collection_20[i][j] * (1-curr_z*stardard_error_20[i][j]) 
                ub_20[i][j] = choice_collection_20[i][j] * (1+curr_z*stardard_error_20[i][j])
    element_indexes_20,perm_collection_20 = find_element_permutation_indexes(n, collection_20)
    products_notin_collection_20 = find_product_notin_assortment(collection_20,n)
    test_result = choose_z_score_for_RUMpred(curr_z,lb_20,ub_20,choice_collection_20,collection_20,perm_collection_20,products_notin_collection_20)
    if test_result[0] >0 :
        target_z_rum = curr_z
    else:
        break
        
        
print('the chosen z value for RUM prediction is ',target_z_rum)

Gurobi optimization status: 2
Optimal solution found!
The z value for RUM prediction is 2.5758293035489004
Gurobi optimization status: 3
No solution found or an error occurred.
the chosen z value for RUM prediction is  2.5758293035489004


In [62]:
target_z_mdm = 0
for curr_z in z_score:
    lb_20 = np.zeros(frequency_collection_20.shape)
    ub_20 = np.zeros(frequency_collection_20.shape)
    for i in range(lb_20.shape[0]):
        for j in range(lb_20.shape[1]):
            if frequency_collection_20[i][j]!=0:
                lb_20[i][j] = choice_collection_20[i][j] * (1-curr_z*stardard_error_20[i][j]) 
                ub_20[i][j] = choice_collection_20[i][j] * (1+curr_z*stardard_error_20[i][j])
    
    test_result = choose_z_score_for_MDMpred(curr_z,lb_20,ub_20,choice_collection_20,collection_20)
    if test_result[0] >0 :
        target_z_mdm = curr_z
    else:
        break
        
        
print('the chosen z value for MDM prediction is ',target_z_mdm)

Optimal solution found!
The z value for MDM prediction is 2.5758293035489004
the chosen z value for MDM prediction is  2.5758293035489004


choose 99% confidence interval 